In [ ]:
!pip install chromadb

In [ ]:
!pip install -U langchain-community langchain langchain-openai langgraph

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.runnables import RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal, Optional

In [ ]:
class State(TypedDict):
    input: str
    intent: Optional[Literal["rag", "llm"]]
    response: Optional[str]
    quality_ok: Optional[bool]

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# 仮のドキュメント群（検索対象）
docs = [
    Document(page_content="LangGraphはLangChain製のLLM向けステートマシングラフライブラリです。"),
    Document(page_content="LangChainはマルチステップ処理を管理するためのフレームワークです。"),
    Document(page_content="RAGは検索と生成を組み合わせた手法です。")
]

# 分割とベクトル化
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)
embedding = OpenAIEmbeddings()

# Chromaによるインデックス作成
vectorstore = Chroma.from_documents(split_docs, embedding)


In [ ]:
# ノード定義
def classify_intent(state):
  query = state["input"]
  if "資料" in query:
    state["intent"] = "rag"
  else:
    state["intent"] = "llm"
  return state

def rag_node(state):
  query = state["input"]
  retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
  retrieved_docs = retriever.invoke(query)
  retrieved_text = retrieved_docs[0].page_content if retrieved_docs else "該当する情報が見つかりませんでした。"
  return {"response": f"[RAG] {retrieved_text}"}

def llm_node(state):
  user_input = state.get("input", "")

  messages = [HumanMessage(content=user_input)]
  response = llm.invoke(messages)
  return {"response": response.content}

def evaluate_answer(state):
  resp = state["response"]
  if " (" in resp:
    # 品質低いと見なす
    state["quality_ok"] = False
  else:
    state["quality_ok"] = True
  return state

def loop_or_end(state):
  return "repeat" if not state["quality_ok"] else "end"


In [ ]:
# --- グラフ構築 ---
workflow = StateGraph(State)
workflow.set_entry_point("classify")
workflow.add_node("classify", RunnableLambda(classify_intent))
workflow.add_node("rag", RunnableLambda(rag_node))
workflow.add_node("llm", RunnableLambda(llm_node))
workflow.add_node("evaluate", RunnableLambda(evaluate_answer))

# 分岐と遷移
workflow.add_conditional_edges("classify", lambda s: s["intent"], {
    "rag": "rag",
    "llm": "llm"
})
workflow.add_edge("rag", "evaluate")
workflow.add_edge("llm", "evaluate")
workflow.add_conditional_edges("evaluate", loop_or_end, {
    "classify": "classify",
    "end": END
})

app = workflow.compile()

In [ ]:
input_state = {"input": "資料をください"}
for s in app.stream(input_state):
  print(s)

In [ ]:
# 必要なパッケージをインストール
!apt-get install -y graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

In [ ]:
from IPython.display import Image

# グラフの描画
Image(app.get_graph().draw_png())